In [29]:
import pandas as pd
from cdcqr.common.config import LOCAL_DATA_DIR, CDQ_REPO_DIR
import os
from scipy.optimize import minimize
from scipy.interpolate import interp1d
from functools import partial
import numpy as np
DF= pd.DataFrame
from importlib import reload
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

import cdcqr.analytics.derivatives.option_utils as optUtil
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime 
import sys
sys.path.append(CDQ_REPO_DIR)
from ct.utils import plot2
from cdcqr.analytics.derivatives.option_utils import errfunc_parabolic_linear6

In [2]:
reload(optUtil)

<module 'cdcqr.analytics.derivatives.option_utils' from 'c:\\core\\github\\cdcqr\\cdcqr\\analytics\\derivatives\\option_utils.py'>

In [3]:
df=pd.read_pickle(os.path.join(LOCAL_DATA_DIR, 'optchain5Min.pkl'))
df_518_1m=pd.read_pickle(os.path.join(LOCAL_DATA_DIR, 'optchain5Min.pkl'))
df_518_3m=pd.read_pickle(os.path.join(LOCAL_DATA_DIR, 'optchain5Min.pkl'))
df_519_1m=pd.read_pickle(os.path.join(LOCAL_DATA_DIR, 'optchain30Min_20210519_1M.pkl'))
df_519_3m=pd.read_pickle(os.path.join(LOCAL_DATA_DIR, 'optchain30Min_20210519_3M.pkl'))
df_8_1m = pd.read_pickle(os.path.join(LOCAL_DATA_DIR, 'optchain1H_202108_1M.pkl'))

In [4]:
df_829_1m=pd.read_pickle(os.path.join(LOCAL_DATA_DIR, 'optchain5Min_20210829_1M.pkl'))
df_830_1m=pd.read_pickle(os.path.join(LOCAL_DATA_DIR, 'optchain5Min_20210830_1M.pkl'))
df_831_1m=pd.read_pickle(os.path.join(LOCAL_DATA_DIR, 'optchain5Min_20210831_1M.pkl'))

In [5]:
df_202104_1H_June=pd.read_pickle(os.path.join(LOCAL_DATA_DIR, 'optchain1H_202104_1M.pkl'))
df_202105_1H_June=pd.read_pickle(os.path.join(LOCAL_DATA_DIR, 'optchain1H_202105_1M.pkl'))

In [6]:
df_20210829_20210831_20220624_5min = pd.read_pickle(os.path.join(LOCAL_DATA_DIR, 'optchain5Min_20210829_20210831_20220624.pkl'))

---------test start----------

In [35]:
res = []

In [14]:
ts = df.index.get_level_values('tm').drop_duplicates()

In [25]:
t = ts[0]
dft = df.loc[[t]].droplevel('tm')
Ts = dft.index.get_level_values('expire').drop_duplicates()
T = Ts[0]
dftT = dft.loc[[T]].droplevel('expire').drop_duplicates()
tau = (T - t) / pd.to_timedelta(1, unit='D') / 365
dftT['logKFtau'] = np.log(dftT.index / dftT['s']) / np.sqrt(tau)
S = dftT['s'].iloc[0]
dftTOTM = dftT[((dftT.ty == 1) & (S < dftT.index)) | ((dftT.ty == -1) & (S > dftT.index))]
dftTOTM.head(2)

,symbol,aiv,ap,av,biv,bp,bv,delta,exch,exchtm,gamma,lpx,mark,markiv,oint,rho,theta,ty,underly,s,vega,tau,logKFtau
strike,,,,,,,,,,,,,,,,,,,,,,,
10000,BTC-25JUN21-10000-P,2.0905,0.0015,0.6,1.821,0.0005,8.1,-0.00391,deribit,2021-05-19 00:29:58.726,0,0.001,0.00092099,1.9593,764.9,-0.21473,-4.23703,-1,BTC-25JUN21,43493.61,1.61378,0.1022260678,-4.596464733
12000,BTC-25JUN21-12000-P,1.9831,0.0025,21.5,1.7417,0.001,117.6,-0.0065,deribit,2021-05-19 00:29:58.728,0,0.0015,0.0015032,1.8396,665.6,-0.35599,-6.26032,-1,BTC-25JUN21,43497.09,2.53957,0.1022260677,-4.026634556


In [31]:
usevega = True
model="errfunc_parabolic_linear6"
prevparams = [0.2, 0.2, 0.2, -0.2] if model[-1] == '4' else [0.2] * int(model[-1])
kwargs={}
kwargs['kleft'] = 1.5
kwargs['kright'] = 2.3

minres = minimize(
    partial(eval(model), usevega=usevega, df=dftTOTM, PRICECALL=False, stats=False, plot=False,
            kleft=kwargs['kleft'], kright=kwargs['kright']), prevparams, method='L-BFGS-B',
    options={'eps': 1e-09, 'maxfun': 100, 'maxiter': 100})

In [50]:
plot=3
dt = pd.to_datetime(t)
expiry_date = T

fig_name = '{}_{}'.format(dt.strftime("%Y%m%d%H%M%S"), expiry_date)

plotres = partial(eval(model), usevega=usevega, df=dftTOTM, PRICECALL=True, plot=plot, info=f't={t} T={T}',
                  kleft=kwargs['kleft'], kright=kwargs['kright'], fig_name=fig_name)(minres.x)

In [51]:
plotres

{'volATM': 1.0549179745,
 'perc': 0.2857142857142857,
 'rr': -0.05386019265499509,
 'conv': 0.04512697977408653,
 'figs': []}

In [43]:
resd = {'t': t, 'T': T, 'volATM': plotres['volATM'], 's': S, 'rr': plotres['rr'], 'conv': plotres['conv'],
        'perc': plotres['perc']}
for i in range(len(minres.x)):
    resd['p' + str(i)] = minres.x[i]
res.append(resd)

In [52]:
### test error function

In [57]:
x = prevparams
usevega=False
useweights=False
df=dftTOTM
PRICECALL=False
plot=False
info=""

res = {}
figs = []
skewleft = x[0]
skewright = x[1]
leftconv = x[2]
rightconv = x[3]
rightwing = x[4]
leftwing = x[5]

kright = kwargs['kright']
kleft = kwargs['kleft']

In [58]:
df['m'] = 0.5 * (df['aiv'] + df['biv'])
S = df['s'].iloc[0]

In [62]:
volATM = interp1d(df.index, df['m'])([S])[0]

In [ ]:
df['logKFtau']

In [ ]:
df.loc[(df.logKFtau > 0) & (df.logKFtau < kright), 'myvol'] = volATM + (
        skewright * df['logKFtau'] + rightconv * df['logKFtau'] ** 2)

In [ ]:
df.loc[(df.logKFtau < 0) & (df.logKFtau > kleft), 'myvol'] = volATM + (
        skewleft * df['logKFtau'] + leftconv * df['logKFtau'] ** 2)

----------test end------------

In [13]:
reload(optUtil)
df = df_519_1m.copy()
df=df[df.symbol.str.contains('BTC')]
df=df.set_index(['tm','expire','strike']).sort_index()
_ = optUtil.volfit(df,plot=3,model="errfunc_parabolic_linear6",kleft=-1.5,kright=2.3, )

--------------sampling time= 2021-05-19 00:00:00 expire= 2021-06-25 08:00:00
prevparams=[0.2, 0.2, 0.2, 0.2, 0.2, 0.2] model=errfunc_parabolic_linear6
optimized params:[-0.16502719 -0.18850574  0.03216582  0.10089888  0.18363569 -0.20551079]
--------------sampling time= 2021-05-19 00:30:00 expire= 2021-06-25 08:00:00
prevparams=[0.2, 0.2, 0.2, 0.2, 0.2, 0.2] model=errfunc_parabolic_linear6
optimized params:[-0.17572048 -0.20228266  0.02357751  0.09491295  0.17020698 -0.18194015]
--------------sampling time= 2021-05-19 01:00:00 expire= 2021-06-25 08:00:00
prevparams=[0.2, 0.2, 0.2, 0.2, 0.2, 0.2] model=errfunc_parabolic_linear6
optimized params:[-0.19762811 -0.20685828  0.0062706   0.09434525  0.15879918 -0.18145017]
--------------sampling time= 2021-05-19 01:30:00 expire= 2021-06-25 08:00:00
prevparams=[0.2, 0.2, 0.2, 0.2, 0.2, 0.2] model=errfunc_parabolic_linear6
optimized params:[-0.1752297  -0.20997553  0.01916198  0.09140906  0.15972021 -0.15401731]
--------------sampling time= 202

In [ ]:
df = df_518_1m.copy()
df=df[df.symbol.str.contains('BTC')]
df=df.set_index(['tm','expire','strike']).sort_index()
optUtil.volfit(df,plot=3,model="errfunc_parabolic_linear6",kleft=-1.5,kright=2.3)

In [ ]:
vfres=volfit(df,plot=0,model='errfunc_parabolic_linear6',kleft=-2,kright=1.5)

In [ ]:
#5Min
dfv=DF(vfres)

dfv['r']=dfv['s'].pct_change()#*dfv['rr'].shift(1)
dfv['r*rrprev']=dfv['r']*dfv['rr'].shift(1)
dfv['r*p0prev']=dfv['r']*dfv['p0'].shift(1)
dfv['r*convprev']=dfv['r']*dfv['conv'].shift(1)

dfv['dv']=dfv['volATM'].diff(1)
dfv.head()
sm.OLS(dfv['dv'],dfv['r'],missing='drop').fit().params

sns.regplot(x='r*p0prev',y='dv',data=dfv).set_title(f"r*p0prev beta={sm.OLS(dfv['dv'],dfv['r*p0prev'],missing='drop').fit().params[0]} corr={dfv.corr()['dv']['r*p0prev']}")
plt.show()
sns.regplot(x='r*rrprev',y='dv',data=dfv).set_title(f"r*rrprev beta={sm.OLS(dfv['dv'],dfv['r*rrprev'],missing='drop').fit().params[0]} corr={dfv.corr()['dv']['r*rrprev']}")
plt.show()
sns.regplot(x='r',y='dv',data=dfv).set_title(f"r beta={sm.OLS(dfv['dv'],dfv['r'],missing='drop').fit().params[0]} corr={dfv.corr()['dv']['r']}")

In [ ]:
DF(vfres).set_index('t')[['p0','p1','p2','p3','p4','p5']].plot()

### 2021 AUG 1H 

In [ ]:
df = df_8_1m.copy()
df=df[df.symbol.str.contains('BTC')]
df=df.set_index(['tm','expire','strike']).sort_index()
optUtil.volfit(df,plot=3,model="errfunc_parabolic_linear6",kleft=-1.5,kright=2.3)

In [ ]:
vfres=volfit(df,plot=0,model='errfunc_parabolic_linear6',kleft=-2,kright=1.5)
DF(vfres).set_index('t')[['p0','p1','p2','p3','p4','p5']].plot()

### 2021 AUG 29 5min 

In [11]:
df = df_829_1m.copy()
df=df[df.symbol.str.contains('BTC')]
df=df.set_index(['tm','expire','strike']).sort_index()
optUtil.volfit(df,plot=3,model="errfunc_parabolic_linear6",kleft=-1.5,kright=2.3)

T=2021-09-24 08:00:00 t=2021-08-29 00:00:00
prevparams=[0.2, 0.2, 0.2, 0.2, 0.2, 0.2] model=errfunc_parabolic_linear6
optimized params:[-0.01167822 -0.01212231  0.07229385  0.05164996  0.22916306 -0.26050212]
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


KeyError: 'expiry_date'

In [ ]:
vfres=volfit(df,plot=0,model='errfunc_parabolic_linear6',kleft=-2,kright=1.5)
DF(vfres).set_index('t')[['p0','p1','p2','p3','p4','p5']].plot()

In [12]:
vfres

NameError: name 'vfres' is not defined

#### 2021 AUG 30 5min 

In [ ]:
df = df_830_1m.copy()
df=df[df.symbol.str.contains('BTC')]
df=df.set_index(['tm','expire','strike']).sort_index()
optUtil.volfit(df,plot=3,model="errfunc_parabolic_linear6",kleft=-1.5,kright=2.3)

In [ ]:
vfres=volfit(df,plot=0,model='errfunc_parabolic_linear6',kleft=-2,kright=1.5)
DF(vfres).set_index('t')[['p0','p1','p2','p3','p4','p5']].plot()

#### 2021 AUG 31 5min 

In [ ]:
df = df_831_1m.copy()
df=df[df.symbol.str.contains('BTC')]
df=df.set_index(['tm','expire','strike']).sort_index()
optUtil.volfit(df,plot=3,model="errfunc_parabolic_linear6",kleft=-1.5,kright=2.3)

In [ ]:
vfres=volfit(df,plot=0,model='errfunc_parabolic_linear6',kleft=-2,kright=1.5)
DF(vfres).set_index('t')[['p0','p1','p2','p3','p4','p5']].plot()

#### 2021 April 1H 

In [ ]:
df = df_202104_1H_June.copy()
df=df[df.symbol.str.contains('BTC')]
df=df.set_index(['tm','expire','strike']).sort_index()
vfres = optUtil.volfit(df,plot=3,model="errfunc_parabolic_linear6",kleft=-1.5,kright=2.3)

DF(vfres).set_index('t')[['p0','p1','p2','p3','p4','p5']].plot()

#### 2021 May 1H 

In [ ]:
df_202105_1H_June[df_202105_1H_June.index>datetime(2021,5,21,0,0)]

In [ ]:
df = df_202105_1H_June.copy()
df=df[df.symbol.str.contains('BTC')]
df=df.set_index(['tm','expire','strike']).sort_index()
vfres = optUtil.volfit(df,plot=3,model="errfunc_parabolic_linear6",kleft=-1.5,kright=2.3)

In [ ]:
df

In [ ]:
DF(vfres).set_index('t')[['p0','p1','p2','p3','p4','p5']].plot()

#### 202104 spike analysis

In [ ]:
df_202104_spikes = pd.read_pickle(os.path.join(LOCAL_DATA_DIR, 'optchain5Min_202104_spike_days.pkl'))

In [ ]:
df = df_202104_spikes.copy()
df=df[df.symbol.str.contains('BTC')]
df=df.set_index(['tm','expire','strike']).sort_index()
vfres = optUtil.volfit(df,plot=3,model="errfunc_parabolic_linear6",kleft=-1.5,kright=2.3)

In [ ]:
df_202104_spikes_18 = df_202104_spikes[(df_202104_spikes['tm']<datetime(2021,4,19,0,0)) & (df_202104_spikes['tm']>=datetime(2021,4,18,0,0))]

In [ ]:
df = df_202104_spikes_18.copy()
df=df[df.symbol.str.contains('BTC')]
df=df.set_index(['tm','expire','strike']).sort_index()
vfres = optUtil.volfit(df,plot=3,model="errfunc_parabolic_linear6",kleft=-1.5,kright=2.3)

In [ ]:
DF(vfres).set_index('t')[['volATM','p0','p1','p2','p3','p4','p5']].plot()

#### 202105 spike analysis


In [ ]:
df_202105_spikes = pd.read_pickle(os.path.join(LOCAL_DATA_DIR, 'optchain5Min_202105_spike_days.pkl'))

In [ ]:
df_20210519_spikes = pd.read_pickle(os.path.join(LOCAL_DATA_DIR, 'optchain1Min_20210519_20211231.pkl'))

In [ ]:
df_20210519_spikes

In [ ]:
df = df_202105_spikes

In [ ]:
df = df_202105_spikes[df_202105_spikes.index<=datetime(2021,5,16,2,0,0)]

In [ ]:
df  = df_202105_spikes[df_202105_spikes['symbol'].str.contains('BTC')].head(500)

In [ ]:
\df

In [ ]:
df=df[df.symbol.str.contains('BTC')]
df=df.set_index(['tm','expire','strike']).sort_index()
vfres = optUtil.volfit(df,plot=3,model="errfunc_parabolic_linear6",kleft=-1.5,kright=2.3)

In [ ]:
vfres

In [ ]:
df.iloc[:3,:]

In [ ]:
DF(vfres).set_index('t')[['volATM','p0','p1','p2','p3','p4','p5']].plot()

### df_20210829_20210831_20220624_5min

In [ ]:
df = df_20210829_20210831_20220624_5min.copy()
df=df[df.symbol.str.contains('BTC')]
df=df.set_index(['tm','expire','strike']).sort_index()
vfres = optUtil.volfit(df,plot=3,model="errfunc_parabolic_linear6",kleft=-1.5,kright=2.3, expiry_date='20220624')

In [ ]:
DF(vfres).set_index('t')[['p0','p1','p2','p3','p4','p5']].iplot()

In [ ]:
DF(vfres).set_index('t')[['p0','p1','p2','p3','p4','p5']].plot()